In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from math import floor
import seaborn as sns
from scipy.stats import spearmanr
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sko.PSO import PSO
from sko import GA
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.datasets import make_classification
from sklearn.preprocessing import PowerTransformer, QuantileTransformer, StandardScaler, MinMaxScaler, FunctionTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, Lasso, LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, RandomizedSearchCV
from sklearn.neighbors import KNeighborsRegressor
from niapy.algorithms.basic import CuckooSearch
from evopreprocess.feature_selection import EvoFeatureSelection
from GP import FeatureTransformation
from PSO import MultiPSO
import random

In [2]:
dataset = pd.read_excel(os.path.join('databases', 'final_dataset.xlsx'), usecols=lambda x: 'Unnamed' not in x)
dataset.drop(columns=['name', 'code'], inplace=True)
y = dataset.pop('pop_growth')

In [3]:
X_train, X_test, y_train, y_test = train_test_split(dataset, y, test_size=0.2, random_state=1)
X_train.reset_index(inplace=True, drop=True)
X_test.reset_index(inplace=True, drop=True)
standard_scaler = StandardScaler()


mm_scaler = MinMaxScaler()

X_train['year'] = mm_scaler.fit_transform(X_train.loc[:, ['year']]).reshape(-1, )
X_test['year'] = mm_scaler.fit_transform(X_test.loc[:, ['year']]).reshape(-1, )
year_train = X_train.pop('year')
year_test = X_test.pop('year')

X_train = pd.DataFrame(data=standard_scaler.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(data=standard_scaler.fit_transform(X_test), columns=X_test.columns)

X_train['year'], X_test['year'] = year_train, year_test


print(X_train.head(5))

   age_dep_young_x  age_dep_old_x  net_migration_x  rural_pop_x  death_rate_x  \
0         0.317331      -0.700878         0.066412    -0.327252     -1.207614   
1        -1.517805       2.214166         0.151152    -0.345429     -0.208437   
2        -0.731006      -0.062341        -1.273974     2.004305     -0.569271   
3        -0.767975       0.671879        -1.149784    -0.077974      0.471101   
4        -0.174441      -0.312642         0.065354    -0.346067     -0.826995   

   fertilty_rate_x  life_exp_female_x  life_exp_male_x  agri_land_x  \
0        -0.092813           0.516137         0.666431    -0.375990   
1        -1.262569           1.379472         1.439999    -0.386290   
2        -1.057964           0.460023         0.407349     2.031557   
3        -0.948879           0.371307        -0.171422     0.609111   
4        -0.557244           0.458653         0.508876    -0.386912   

   mortality_male_x  ...  life_exp_female_y  life_exp_male_y  agri_land_y  \
0        

In [4]:
ridge_basic = Ridge(max_iter=5000, random_state=1)
ridge_grid = {"alpha": np.logspace(-8, 8, 100)}

lasso_basic = Lasso(max_iter=5000, random_state=1)
lasso_grid = {"alpha": (np.logspace(-8, 8, 100)), }

knn_basic = KNeighborsRegressor()
knn_grid = {'n_neighbors' : list(range(3, 21)),
            'weights' : ['uniform','distance'],
            'metric' : ['minkowski','euclidean','manhattan']
}

dtr_basic = DecisionTreeRegressor(random_state=1)
dtr_grid = {"max_features": np.arange(0.7, 1.01, 0.1), 
            "max_depth": range(10, 21, 2), 
             "min_samples_split":np.arange(0.02, 0.055, 0.01), 
             "min_samples_leaf": range(1, 21, 2)  
}

rfr_basic = RandomForestRegressor(random_state=1)
rfr_grid = {"n_estimators": [100], 
             "max_depth": range(10, 21, 2), 
             "min_samples_split": np.arange(0.02, 0.041, 0.01), 
             "min_samples_leaf": range(1, 7, 2)
}

In [5]:
def train_model(model, X_train, X_test, y_train, y_test, params_grid):
    cv = KFold(n_splits=5, shuffle=True, random_state=1)
    our_model = GridSearchCV(model, param_grid=params_grid, cv=cv, scoring="neg_mean_absolute_error").fit(X_train, y_train).best_estimator_
    our_model.fit(X_train, y_train)
    y_pred = our_model.predict(X_test)
    return mean_squared_error(y_test, y_pred)

In [10]:
rfr1_score = train_model(rfr_basic, X_train, X_test, y_train, y_test, rfr_grid)
print(rfr1_score)

: 

In [13]:
ridge1_score = train_model(ridge_basic, X_train, X_test, y_train, y_test, ridge_grid)
print(ridge1_score)
lasso1_score = train_model(lasso_basic, X_train, X_test, y_train, y_test, lasso_grid)
print(lasso1_score)
knn1_score = train_model(knn_basic, X_train, X_test, y_train, y_test, knn_grid)
print(knn1_score)
dtr1_score = train_model(dtr_basic, X_train, X_test, y_train, y_test, dtr_grid)
print(dtr1_score)
rfr1_score = train_model(rfr_basic, X_train, X_test, y_train, y_test, rfr_grid)
print(rfr1_score)

0.02238093380770618


c:\Users\yassi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.495e+01, tolerance: 1.375e+00
  model = cd_fast.enet_coordinate_descent(
c:\Users\yassi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.685e+01, tolerance: 1.393e+00
  model = cd_fast.enet_coordinate_descent(
c:\Users\yassi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the sca

0.022156994509075406
0.28987972077385665
0.6723382983760418


: 

Here we are preprocessing the data set by our PSO algorithm

In [6]:
pso = MultiPSO(dataset)

data = pso.eliminate_multicollinearity(dataset)
data.reset_index(drop=True, inplace=True)

X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=1)
X_train.reset_index(inplace=True, drop=True)
X_test.reset_index(inplace=True, drop=True)
standard_scaler = StandardScaler()


mm_scaler = MinMaxScaler()

X_train['year'] = mm_scaler.fit_transform(X_train.loc[:, ['year']]).reshape(-1, )
X_test['year'] = mm_scaler.fit_transform(X_test.loc[:, ['year']]).reshape(-1, )
year_train = X_train.pop('year')
year_test = X_test.pop('year')

X_train = pd.DataFrame(data=standard_scaler.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(data=standard_scaler.fit_transform(X_test), columns=X_test.columns)

X_train['year'], X_test['year'] = year_train, year_test


print(X_train.head(5))

c:\Users\yassi\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


          0         1         2         3         4         5         6  \
0  0.315069 -0.700878  0.066412 -0.327252 -1.207614 -0.811367 -0.666431   
1 -1.516875  2.214166  0.151152 -0.345429 -0.208437 -1.362524 -1.439999   
2 -0.733164 -0.062341 -1.273974  2.004305 -0.569271 -0.502640 -0.407349   
3 -0.769301  0.671879 -1.149784 -0.077974  0.471101  0.969871  0.171422   
4 -0.176721 -0.312642  0.065354 -0.346067 -0.826995 -0.270779 -0.508876   

          7         8         9        10        11        12        13  \
0 -0.192776 -0.125909  0.306919  0.230690 -0.592423  0.066412 -0.327252   
1 -0.164633  2.899800  0.001451  0.165889 -1.223842  0.151152 -0.345429   
2  0.367381 -0.414306  0.232819  0.320922 -0.657238 -1.273974  2.004305   
3 -0.113093 -0.403713 -2.545447 -2.422313 -0.334923 -1.149784 -0.077974   
4 -0.231175 -0.361977 -0.866044 -0.842009 -0.307029  0.065354 -0.346067   

         14        15        16      year  
0 -0.092813 -0.516137 -0.192776  0.966667  
1 -1.26256

In [ ]:
def preprocess_model(model, X_train, X_test, y_train, y_test, params_grid):
    evo = EvoFeatureSelection(evaluator=model, random_seed=1, optimizer=CuckooSearch, n_runs=5, n_folds=3)
    X_new_train = evo.fit_transform(X_train, y_train)
    X_new_test = evo.transform(X_test)
    transformer = FeatureTransformation()
    X_new_train = transformer.fit_transform(X_new_train, y_train)
    X_new_test = transformer.transform(X_new_test)
    X_new_train = X_new_train.rename(columns=lambda x: str(x))
    X_new_test = X_new_test.rename(columns=lambda x: str(x))
    print(X_new_train.head(5))
    return train_model(model, X_new_train, X_new_test, y_train, y_test, params_grid)

In [10]:
ridge2_score = preprocess_model(ridge_basic, X_train, X_test, y_train, y_test, ridge_grid)
print(ridge2_score)
lasso2_score = preprocess_model(lasso_basic, X_train, X_test, y_train, y_test, lasso_grid)
print(lasso2_score)
knn2_score = preprocess_model(knn_basic, X_train, X_test, y_train, y_test, knn_grid)
print(knn2_score)
dtr2_score = preprocess_model(dtr_basic, X_train, X_test, y_train, y_test, dtr_grid)
print(dtr2_score)
rfr2_score = preprocess_model(rfr_basic, X_train, X_test, y_train, y_test, rfr_grid)
print(rfr2_score)

   bc(0**2**2**2)      1**2  recip(recip(cos(2)))    tan(3)**4      4**3  \
0        0.311192  0.004411              0.946929     0.184370 -0.053153   
1        0.311191  0.022847              0.940930  1534.662164 -0.057642   
2        0.311216  1.623009             -0.420059     0.053013  8.384691   
3        0.311191  1.322003              0.996962     0.000387  0.225990   
4        0.311192  0.004271              0.940714     0.077072 -0.057921   

   sin(5)**2**4  yj(sin(cos(6)))   sqrt(7)  cos(sqrt(8))  sqrt(sin(9))**3  \
0      0.039412         0.631504 -0.354821      0.867589         0.081212   
1      0.751831        -1.031904  1.702880      0.959543         0.122653   
2      0.008490         0.477253 -0.643666      0.864913         0.211748   
3      0.000190         1.072420 -0.635420     -0.008528        -0.615722   
4      0.000050         1.112873 -0.601644      0.601103        -0.625604   

   sin(cos(10**2)**2)        11  sqrt(recip(sqrt(12))**1)  
0            0.70149

c:\Users\yassi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_ridge.py:204: LinAlgWarning: Ill-conditioned matrix (rcond=6.05139e-31): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
c:\Users\yassi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_ridge.py:204: LinAlgWarning: Ill-conditioned matrix (rcond=1.57576e-31): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
c:\Users\yassi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_ridge.py:204: LinAlgWarning: Ill-conditioned matrix (rcond=1.35492e-31): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
c:\Users\yassi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_ridge.py:204: LinAlgWarning: Ill-conditioned matrix (rcond=1.32416e-31): result may not be accurate.
  return linalg.solve

1.544345839491175
   yj(sin(0)**4)  recip(sqrt(1**2))  sqrt(yj(yj(2)))
0      -0.484179           1.723917         1.262268
1      -0.460276           0.708032         0.821997
2       2.509626           2.235186        -0.246329
3       2.509199           7.176017        -0.427829
4      -0.484237           2.062145         0.629153
1.7637660278320546


: 